# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import requests
import time
import re
from bs4 import BeautifulSoup

In [8]:
input_keyword = "量子電腦"  

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  
utf8_url = utf8_url.replace("\\X", "%")                 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     

root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

量子電腦: %E9%87%8F%E5%AD%90%E9%9B%BB%E8%85%A6
/wiki/%E9%87%8F%E5%AD%90%E9%9B%BB%E8%85%A6


In [9]:
urls = []
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    
        
        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    print("外部連結: [%s] %s" % (a_keyword, a_link))
                    urls.append(a_link)

In [10]:
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E9%87%8F%E5%AD%90%E9%9B%BB%E8%85%A6 (量子電腦)
外部連結: [通用計算] /wiki/%E9%80%9A%E7%94%A8%E8%AE%A1%E7%AE%97
外部連結: [電子電腦] /wiki/%E7%94%B5%E5%AD%90%E8%AE%A1%E7%AE%97%E6%9C%BA
外部連結: [量子位元] /wiki/%E9%87%8F%E5%AD%90%E4%BD%8D%E5%85%83
外部連結: [馬約拉納費米子] /wiki/%E9%A9%AC%E7%BA%A6%E6%8B%89%E7%BA%B3%E8%B4%B9%E7%B1%B3%E5%AD%90
外部連結: [反粒子] /wiki/%E5%8F%8D%E7%B2%92%E5%AD%90
外部連結: [演算法] /wiki/%E6%BC%94%E7%AE%97%E6%B3%95
外部連結: [電腦科學] /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%A7%91%E5%AD%A6
外部連結: [電腦] /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA
外部連結: [理察·費曼] /wiki/%E7%90%86%E6%9F%A5%E5%BE%B7%C2%B7%E8%B2%BB%E6%9B%BC
外部連結: [量子圖靈機] /wiki/%E9%87%8F%E5%AD%90%E5%9C%96%E9%9D%88%E6%A9%9F
外部連結: [量子] /wiki/%E9%87%8F%E5%AD%90
外部連結: [希爾伯特空間] /wiki/%E5%B8%8C%E7%88%BE%E4%BC%AF%E7%89%B9%E7%A9%BA%E9%96%93
外部連結: [計算機] /wiki/%E8%A8%88%E7%AE%97%E6%A9%9F
外部連結: [積體電路] /wiki/%E7%A9%8D%E9%AB%94%E9%9B%BB%E8%B7%AF
外部連結: [彼得·秀爾] /wiki/%E5%BD%BC%E5%BE%97%C2%B7%E7%A7%80%E7%88%BE
外部連結: [量子質因數分解演算法] /wiki/%E7%A7%80%E7%88%BE%E6%BC%94%E7%AE%97

In [11]:
startTime = time.time()

for url in urls:
    target = 'https://zh.wikipedia.org' + url
    requests.get(target)

finishTime = time.time()
print(finishTime - startTime) # 正常情況的爬蟲所需時間

76.59080004692078


In [12]:
import _thread

startTime = time.time()

for url in urls:
    target = 'https://zh.wikipedia.org' + url
    _thread.start_new_thread( requests.get, (target, ) )

finishTime = time.time()
print(finishTime - startTime) # 利用 thread 的爬蟲所需時間

0.0733332633972168
